In [1]:
import tensorflow as tf
import pandas as pd
import regex as re # Regular Expressions

2023-09-28 20:42:12.363298: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-28 20:42:13.348072: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
train_df = pd.read_csv('train.csv')
# labels = train_df.pop('label')
# train_ds = tf.data.Dataset.from_tensor_slices((list(train_df['text']), labels))

x_train = train_df['text'].apply(lambda text: re.sub('<br />', ' ', text)).to_numpy()
y_train = train_df['label'].to_numpy()

test_df = pd.read_csv('test.csv')
# labels = test_df.pop('label')
# test_ds = tf.data.Dataset.from_tensor_slices((list(test_df['text']), labels))
x_test = test_df['text'].apply(lambda text: re.sub('<br />', ' ', text)).to_numpy()
y_test = test_df['label'].to_numpy()

valid_df = pd.read_csv('valid.csv')
# labels = test_df.pop('label')
# test_ds = tf.data.Dataset.from_tensor_slices((list(test_df['text']), labels))
x_valid = valid_df['text'].apply(lambda text: re.sub('<br />', ' ', text)).to_numpy()
y_valid = valid_df['label'].to_numpy()
print(tf.config.list_physical_devices('GPU'))
# print(x_train, y_train)
# print(x_test, y_test)
# print(list(train_ds.as_numpy_iterator())[0])
# print(test_ds)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2023-09-28 20:42:15.345059: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-28 20:42:15.466729: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-28 20:42:15.467007: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [3]:
vec_layer = tf.keras.layers.TextVectorization(
    max_tokens=10000,
    output_sequence_length=250,
    standardize="lower_and_strip_punctuation"
)
vec_layer.adapt(x_train)
print(vec_layer.get_vocabulary())

2023-09-28 20:42:19.075205: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-28 20:42:19.075381: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-28 20:42:19.075483: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

['', '[UNK]', 'the', 'and', 'a', 'of', 'to', 'is', 'in', 'it', 'i', 'this', 'that', 'was', 'as', 'for', 'with', 'movie', 'but', 'film', 'on', 'you', 'not', 'are', 'his', 'have', 'be', 'he', 'one', 'its', 'at', 'all', 'by', 'an', 'they', 'from', 'who', 'so', 'like', 'just', 'or', 'her', 'about', 'if', 'out', 'has', 'some', 'there', 'what', 'good', 'more', 'when', 'very', 'my', 'even', 'no', 'up', 'would', 'time', 'she', 'only', 'really', 'which', 'see', 'their', 'story', 'were', 'had', 'can', 'me', 'well', 'much', 'than', 'we', 'been', 'get', 'will', 'other', 'also', 'bad', 'do', 'into', 'great', 'people', 'because', 'how', 'him', 'most', 'first', 'dont', 'them', 'then', 'movies', 'films', 'make', 'made', 'way', 'too', 'could', 'any', 'characters', 'after', 'think', 'watch', 'seen', 'many', 'being', 'character', 'two', 'never', 'love', 'acting', 'plot', 'did', 'where', 'little', 'best', 'know', 'show', 'life', 'ever', 'does', 'your', 'better', 'off', 'end', 'over', 'still', 'these', 'sc

In [4]:
# def vectorize_text(text, label):
#   text = tf.expand_dims(text, -1)
#   return vec_layer(text), label

# final_train_ds = train_ds.map(vectorize_text)
# final_test_ds = test_ds.map(vectorize_text)
# print(list(final_train_ds.as_numpy_iterator())[0])

In [5]:
embedding_dim = 32
max_features = 100000
model = tf.keras.Sequential([
    vec_layer,
    tf.keras.layers.Embedding(max_features + 1, embedding_dim, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(
        128
    )),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVe  (None, 250)               0         
 ctorization)                                                    
                                                                 
 embedding (Embedding)       (None, 250, 32)           3200032   
                                                                 
 bidirectional (Bidirection  (None, 256)               164864    
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense (Dense)               (None, 1)                 257       
                                                                 
Total params: 3365153 (12.84 MB)
Trainable params: 33651

In [6]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer='adam',
              metrics=['accuracy']
)


In [7]:
epochs = 10
history = model.fit(
    x_train,
    y_train,
    epochs=epochs,
    validation_data=(x_valid,y_valid)
)


Epoch 1/10


2023-09-28 20:42:28.032351: W tensorflow/core/common_runtime/type_inference.cc:339] Type inference failed. This indicates an invalid graph that escaped type checking. Error message: INVALID_ARGUMENT: expected compatible input types, but input 1:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_INT32
    }
  }
}
 is neither a subtype nor a supertype of the combined inputs preceding it:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_FLOAT
    }
  }
}

	for Tuple type infernce function 0
	while inferring type of node 'cond_40/output/_23'
2023-09-28 20:42:28.256210: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-09-28 20:42:28.413912: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fd13190d8b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-09-28 20:

1250/1250 [==============================] - 83s 62ms/step - loss: 0.5834 - accuracy: 0.6933 - val_loss: 0.5051 - val_accuracy: 0.7288
Epoch 2/10
1250/1250 [==============================] - 38s 30ms/step - loss: 0.5413 - accuracy: 0.7286 - val_loss: 0.5601 - val_accuracy: 0.7430
Epoch 3/10
1250/1250 [==============================] - 38s 31ms/step - loss: 0.4195 - accuracy: 0.8051 - val_loss: 0.3162 - val_accuracy: 0.8650
Epoch 4/10
1250/1250 [==============================] - 36s 29ms/step - loss: 0.2731 - accuracy: 0.8931 - val_loss: 0.2953 - val_accuracy: 0.8814
Epoch 5/10
1250/1250 [==============================] - 35s 28ms/step - loss: 0.2234 - accuracy: 0.9140 - val_loss: 0.2841 - val_accuracy: 0.8838
Epoch 6/10
1250/1250 [==============================] - 33s 27ms/step - loss: 0.1890 - accuracy: 0.9291 - val_loss: 0.2829 - val_accuracy: 0.8872
Epoch 7/10
1250/1250 [==============================] - 32s 26ms/step - loss: 0.1590 - accuracy: 0.9427 - val_loss: 0.3498 - val_accura

In [8]:
model.evaluate(
    x_test,
    y_test
)

157/157 [==============================] - 2s 10ms/step - loss: 0.4141 - accuracy: 0.8748


[0.4141148328781128, 0.8748000264167786]

In [15]:
model.predict(
    ["I love this movie", "Absolute garbage of a movie, hope the directors never work again"]
)

1/1 [==============================] - 0s 27ms/step


array([[0.9012827],
       [0.0045063]], dtype=float32)

In [19]:
model.predict(
    ["Truly a cinematic masterpiece, the actors should be awarded medals for this amazing performance"]
)

1/1 [==============================] - 0s 34ms/step


array([[0.91112417]], dtype=float32)